# Convolutional Neural Network - Gap / Char Classification
Using TensorFlow

Also need to convert images in folder into csv data file

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

# Creating CSV
import glob
import csv

print("OpenCV: " + cv2.__version__)
print("Numpy: " + np.__version__)
print("TensorFlow: " + tf.__version__)

OpenCV: 3.1.0-dev
Numpy: 1.12.0
TensorFlow: 1.0.0-rc1


In [2]:
def implt(img, cmp=None, t=''):
    """ Show image using plt """
    plt.imshow(img, cmap=cmp)
    plt.title(t)
    plt.show()

## Load Images and Lables in CSV

In [3]:
# 755 chars and 306 gaps images, dimension 30x60 = 1800
# Or read_csv() form pandas
images = np.genfromtxt('data/gapdet/data.csv', delimiter=',')
labels = np.genfromtxt('data/gapdet/labels.csv', delimiter=',')
labels = np.reshape(labels, (len(labels), 1))

print("CSV data files loaded.")
print("Number of images: " + str(len(images)))

div = int(0.8 * len(images))

trainData = images[0:div]
trainLabels = labels[0:div]
evalData = images[div:]
evalLabels = labels[div:]
print("Training images: %g" % div)

CSV data files loaded.
Number of images: 3413
Training images: 2730


# Create classifier

### Simoid

In [4]:
# Place holders for data and labels
x = tf.placeholder(tf.float32, [None, 1800], name = "x")
y_ = tf.placeholder(tf.float32, [None, 1], name = "y_")

# Learned variables
W = tf.Variable(tf.random_normal([1800, 1]), name = "W")
b = tf.Variable(tf.random_normal([1]), name = "b")

y = tf.matmul(x, W) + b

# Training Model
cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=y, name="Cost"))
train_step = tf.train.AdamOptimizer(0.1).minimize(cross_entropy)

# Evaluating model
correct_prediction = tf.equal(tf.sigmoid(y) , y_)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# For saving and using model
activation = tf.sigmoid(tf.matmul(x, W) + b)
tf.add_to_collection("act", activation)

for i in range(5000):
#     sess.run(train_step, feed_dict={x: images, y_: labels})
    sess.run(train_step, feed_dict={x: trainData, y_: trainLabels})
    if i % 500 == 0:
        cost = cross_entropy.eval(feed_dict={x: trainData, y_: trainLabels})
        acc = accuracy.eval(feed_dict={x: trainData, y_: trainLabels})
        print("%d #  Cost: %g  Accuracy: %g" % (i, cost, acc))


# Saving model
# tf.train.export_meta_graph('models/CGClassifier')
saver = tf.train.Saver()
saver.save(sess, 'models/CGClassifier')
saver.export_meta_graph('models/CGClassifier.meta')


print("Eval accuracy: %g" % sess.run(accuracy, feed_dict={x: evalData, y_: evalLabels}))
print("Total accuracy: %g" % sess.run(accuracy, feed_dict={x: images, y_: labels}))

sess.close()

0 #  Cost: 1109.2  Accuracy: 0.613919
500 #  Cost: 2.98406e-06  Accuracy: 0.963004
1000 #  Cost: 5.92139e-08  Accuracy: 0.962637
1500 #  Cost: 2.73716e-08  Accuracy: 0.962637
2000 #  Cost: 1.65529e-08  Accuracy: 0.962637
2500 #  Cost: 1.11373e-08  Accuracy: 0.962271
3000 #  Cost: 7.95373e-09  Accuracy: 0.962271
3500 #  Cost: 5.89079e-09  Accuracy: 0.962637
4000 #  Cost: 4.45346e-09  Accuracy: 0.962637
4500 #  Cost: 3.43203e-09  Accuracy: 0.962637
Eval accuracy: 0.68082
Total accuracy: 0.906241


### Sofmax

In [5]:
def sofmaxLabels(lab):
    """ Transform data for sofmax classification """
    newLab = np.zeros((len(lab), 2), dtype=np.int8)
    for i in range(len(lab)):
        newLab[i][int(lab[i][0])] = 1
    return newLab

llabels = sofmaxLabels(labels)
trainLabels = llabels[0:div]
evalLabels = llabels[div:]

In [6]:
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, [None, 1800])
y_ = tf.placeholder(tf.float32, [None, 2])

W = tf.Variable(tf.random_normal([1800, 2]))
b = tf.Variable(tf.random_normal([2]))

y = tf.matmul(x,W) + b

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.AdamOptimizer(0.1).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.global_variables_initializer())

for i in range(5000):
    sess.run(train_step, feed_dict={x: trainData, y_: trainLabels})
    if i % 500 == 0:
        cost = cross_entropy.eval(feed_dict={x: trainData, y_: trainLabels})
        acc = accuracy.eval(feed_dict={x: trainData, y_: trainLabels})
        print("%d #  Cost: %g  Accuracy: %g" % (i, cost, acc))

    

print("Eval accuracy: %g" % accuracy.eval(feed_dict={x: evalData, y_: evalLabels}))
print("Total accuracy: %g" % accuracy.eval(feed_dict={x: images, y_: llabels}))

sess.close()

0 #  Cost: 3836.67  Accuracy: 0.706593
500 #  Cost: 0.511161  Accuracy: 0.998901
1000 #  Cost: 7.46693e-09  Accuracy: 1
1500 #  Cost: 4.6723e-09  Accuracy: 1
2000 #  Cost: 3.31864e-09  Accuracy: 1
2500 #  Cost: 2.57632e-09  Accuracy: 1
3000 #  Cost: 2.00865e-09  Accuracy: 1
3500 #  Cost: 1.61566e-09  Accuracy: 1
4000 #  Cost: 1.22266e-09  Accuracy: 1
4500 #  Cost: 1.00433e-09  Accuracy: 1
Eval accuracy: 0.707174
Total accuracy: 0.941401


### Convulation Neural Network

In [9]:
sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, [None, 1800])
y_ = tf.placeholder(tf.float32, [None, 2])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x_image = tf.reshape(x, [-1, 30, 60, 1])

# 1. Layer
W_conv1 = weight_variable([5, 5, 1, 12])
b_conv1 = bias_variable([12])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# 2. Layer
W_conv2 = weight_variable([5, 5, 12, 24])
b_conv2 = bias_variable([24])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Connected layer
W_fc1 = weight_variable([8 * 15 * 24, 512])
b_fc1 = bias_variable([512])

h_pool2_flat = tf.reshape(h_pool2, [-1, 8*15*24])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Output layer
W_fc2 = weight_variable([512, 2])
b_fc2 = bias_variable([2])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


# Train
# Stacking on 18029.2 cost
# cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv + 1e-10))
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()

for i in range(300):
    if i%10 == 0:
        cost = cross_entropy.eval(feed_dict={x: trainData, y_: trainLabels, keep_prob: 1.0})
        acc = accuracy.eval(feed_dict={x: trainData, y_: trainLabels, keep_prob: 1.0})
        print("%d #  Cost: %g  Accuracy: %g" % (i, cost, acc))
        
    if i%50 == 0:
        saver.save(sess, 'models/CNN-CG')
        saver.export_meta_graph('models/CNN-CG.meta')

    train_step.run(feed_dict={x: trainData, y_: trainLabels, keep_prob: 0.5})

print("Eval accuracy %g" % accuracy.eval(feed_dict={x: evalData, y_: evalLabels, keep_prob: 1.0}))
print("Total accuracy: %g" % accuracy.eval(feed_dict={x: images, y_: llabels, keep_prob: 1.0}))

sess.close()

0 #  Cost: 73.5318  Accuracy: 0.504396
10 #  Cost: 44.7366  Accuracy: 0.671429
20 #  Cost: 30.4433  Accuracy: 0.717582
30 #  Cost: 25.0341  Accuracy: 0.755678
40 #  Cost: 18.3042  Accuracy: 0.789377
50 #  Cost: 12.9228  Accuracy: 0.816117
60 #  Cost: 9.94421  Accuracy: 0.835165
70 #  Cost: 8.04221  Accuracy: 0.853846
80 #  Cost: 6.33955  Accuracy: 0.872894
90 #  Cost: 4.87928  Accuracy: 0.889377
100 #  Cost: 4.07447  Accuracy: 0.899634
110 #  Cost: 3.05077  Accuracy: 0.914286
120 #  Cost: 2.30543  Accuracy: 0.927839
130 #  Cost: 1.98584  Accuracy: 0.936996
140 #  Cost: 1.43127  Accuracy: 0.949451
150 #  Cost: 1.06711  Accuracy: 0.957509
160 #  Cost: 0.965051  Accuracy: 0.962271
170 #  Cost: 0.634639  Accuracy: 0.968498
180 #  Cost: 0.506358  Accuracy: 0.973626
190 #  Cost: 0.351714  Accuracy: 0.978388
200 #  Cost: 0.253768  Accuracy: 0.984982
210 #  Cost: 0.189605  Accuracy: 0.987546
220 #  Cost: 0.14456  Accuracy: 0.99011
230 #  Cost: 0.109145  Accuracy: 0.991941
240 #  Cost: 0.079102